In [2]:
#Add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from radiomics import featureextractor
from tqdm import tqdm
from IPython.display import clear_output

#Import paths and patients classes
import notebooks.utils as utils

In [3]:
def substract(im_base, im_restant, x_trans, y_trans, pat_num, sitk_im=False, reference_path=None, save=False):
    """given base and restant images, substract restant from base and return the result
    Minimum value of substraction is shifted to 0 to become uint16 image

    Args:
        im_base (array): base image
        im_restant (array): restant image
        sitk_im (bool, optional): if True, the image is a sitk obj using SET as reference image. Defaults to False.
        reference_path (str, optional): path to SET reference image. Defaults to None.

    Returns:
        array or sitk image: uint16 image with the substraction
    """
    #Subsract using im_base as reference and transform to int32 for negative values
    im_substraction = (im_base.astype(np.int32) - im_restant.astype(np.int32))
    #shift intensities to be uint16
    im_substraction = (im_substraction + 33152).astype(np.uint16) #save with original dtype
    utils.min_max(im_substraction, show=True)

    if save:
        utils.save_as_nifti(im_substraction,
                            filename=str(repo_path / f'data/substracted_images/Pat_{pat_num}_CMC-SMC_x{x_trans}_y{y_trans}.tif'),
                            reference_path=str(reference_path))
    if sitk_im: #in case of sitk image
        reference_image = sitk.ReadImage(reference_path)
        im_substraction = sitk.GetImageFromArray(im_substraction)
        im_substraction.SetOrigin(reference_image.GetOrigin())
        im_substraction.SetSpacing(reference_image.GetSpacing())
        im_substraction.SetDirection(reference_image.GetDirection())
        return im_substraction
    #else return array
    return im_substraction

def extractor_settings(param_path, show=False):
    """set extraction settings for pyradiomics

    Args:
        param_path (str): relative path of parameter file
        show (bool, optional): if printing setting or not. Defaults to False.

    Returns:
        obj: extractor of pyradiomics
    """
    extractor = featureextractor.RadiomicsFeatureExtractor(str(repo_path /param_path))
    if show:
        print('Extraction parameters:\n\t', extractor.settings)
        print('Enabled filters:\n\t', extractor.enabledImagetypes)
        print('Enabled features:\n\t', extractor.enabledFeatures)
    return extractor

In [5]:
# read no contrast image
no_contrast_path = repo_path / 'data/images/pre-SET/Pat_02_RCC_SMC.tif'
no_contrast = sitk.GetArrayFromImage(sitk.ReadImage(str(no_contrast_path)))
# min and max
print(f'min and max of no contrast image: {no_contrast.min()}, {no_contrast.max()}')

min and max of no contrast image: 11341, 14659
